In [88]:
#importing libraries for emails
import imaplib
import base64
import email
import getpass #for entering password securely
#importing boto3 module to connect to aws services
import boto3

In [89]:
#using Amazon s3
s3 = boto3.resource('s3')

In [91]:
#to input email_id and password to access emails
email_user=input('Email:')
email_pass=getpass.getpass(prompt='Password: ', stream=None) 

Email:agrawalaastha798@gmail.com
Password: ········


In [92]:
#making connection with the host over an SSL encrypted socket

mail=imaplib.IMAP4_SSL("imap.gmail.com",993)

In [93]:
#login into the account


try:
    mail.login(email_user, email_pass)
except imaplib.IMAP4.error:
    print ("LOGIN FAILED!!!")

In [94]:
#listing the folders
mail.list()

('OK',
 [b'(\\HasNoChildren) "/" "INBOX"',
  b'(\\HasChildren \\Noselect) "/" "[Gmail]"',
  b'(\\All \\HasNoChildren) "/" "[Gmail]/All Mail"',
  b'(\\Drafts \\HasNoChildren) "/" "[Gmail]/Drafts"',
  b'(\\HasNoChildren \\Important) "/" "[Gmail]/Important"',
  b'(\\HasNoChildren \\Sent) "/" "[Gmail]/Sent Mail"',
  b'(\\HasNoChildren \\Junk) "/" "[Gmail]/Spam"',
  b'(\\Flagged \\HasNoChildren) "/" "[Gmail]/Starred"',
  b'(\\HasNoChildren \\Trash) "/" "[Gmail]/Trash"'])

In [95]:
#to select inbox folder to receive mail
mail.select('Inbox')

('OK', [b'6'])

In [100]:
#to search the mail


t, data = mail.search(None, 'ALL')

id_list = data[0].split()


In [101]:
#to fetch received mail


num=id_list[-1]

typ, data = mail.fetch(num, '(RFC822)' )
raw_email = data[0][1]


# converts byte literal to string removing b''
raw_email_string = raw_email.decode('utf-8')
email_message = email.message_from_string(raw_email_string)


In [ ]:
#for listing files from s3 bucket

l=[]
my_bucket = s3.Bucket('csvfilesfromemails')

for file in my_bucket.objects.all():
    l.append(file.key)

In [104]:
for part in email_message.walk():
        
        if part.get_content_maintype() == 'multipart':
            continue
        if part.get('Content-Disposition') is None:
            continue
        #finding file names from part
        fileName = part.get_filename()
        #splitting file names into a list
        file_lists=re.split('/n',fileName)
        print(file_lists)
        
        for file in file_lists:
            
            if file.endswith('.csv'): #finding whether file is csv file or not
                if file in l:
                    print('File already exists.')
                else:
                    # The object does not exist.Therefore, uploading the file in s3.
                    data = part.get_payload(decode=True)
                    s3.Bucket('csvfilesfromemails').put_object(Key=file, Body=data)
                    print('File uploaded successfully.')
      
            else:
                print('Unwanted file.')
  

['Apply.json']
Unwanted file.
['College.json']
Unwanted file.
['ContainsNull.csv']
File already exists.
['Automating process of sending csv file from email to aws s3.ipynb']
Unwanted file.
